<a href="https://colab.research.google.com/github/S-Mike-Eng/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#########  Sentiment Analysis using LSTM model First GOP Debate Twitter Sentiment About this Dataset This data originally came from Crowdflower's Data for Everyone library ##########

In [ ]:
!git clone https://github.com/sanjay-raghu/sentiment-analysis-using-LSTM-keras

Cloning into 'sentiment-analysis-using-LSTM-keras'...
remote: Enumerating objects: 25, done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 25
Unpacking objects: 100% (25/25), done.


In [ ]:
!ls

sample_data  sentiment-analysis-using-LSTM-keras


In [ ]:
import os
os.chdir('sentiment-analysis-using-LSTM-keras')

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,GRU,Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re
import pandas as pd


In [ ]:
data = pd.read_csv('input/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [ ]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [ ]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
# removing special chars
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#
data.head(5)

In [ ]:
type(data['text'])

pandas.core.series.Series

In [ ]:
data

In [ ]:
print(data[ data['sentiment'] == 'Neutral'].size)

6284


In [ ]:
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
# print(data[ data['sentiment'] == 'Neutral'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt','')
data.head(5)

In [ ]:
data['text']

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         359,  120,    1,  692,    2,   39,   58,  234,   37,  207,    6,
         172, 1745,   12, 1308, 1394,  733],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          16,  281,  249,    5,  809,  102,  170,   26,  134,    6,    1,
         171,   12,    2,  231,  713,   17]], dtype=int32)

In [ ]:
###################################  LSTM  net  #################################################################

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2 ,return_sequences=True))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 28, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 28, 196)           254800    
                                                                 
 lstm_9 (LSTM)               (None, 196)               308112    
                                                                 
 dense_3 (Dense)             (None, 3)                 591       
                                                                 
Total params: 819,503
Trainable params: 819,503
Non-trainable params: 0
________________________________________________

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
Y

array([[0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]], dtype=uint8)

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8583, 28) (8583, 2)
(2146, 28) (2146, 2)


In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size,validation_data=(X_test,Y_test), verbose = 1)

In [ ]:
###################################  convolutional  net  #################################################################

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [ ]:
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 28, 128)           256000    
                                                                 
 conv1d_1 (Conv1D)           (None, 28, 32)            12320     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 14, 32)           0         
 1D)                                                             
                                                                 
 lstm_10 (LSTM)              (None, 100)               53200     
                                                                 
 dense_4 (Dense)             (None, 2)                 202       
                                                                 
Total params: 321,722
Trainable params: 321,722
Non-trainable params: 0
________________________________________________

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size,validation_data=(X_test,Y_test), verbose = 1)

Epoch 1/15
269/269 [==============================] - 8s 28ms/step - loss: 0.1500 - accuracy: 0.9389 - val_loss: 0.5900 - val_accuracy: 0.8281
Epoch 2/15
269/269 [==============================] - 6s 22ms/step - loss: 0.1098 - accuracy: 0.9570 - val_loss: 0.6591 - val_accuracy: 0.8383
Epoch 3/15
269/269 [==============================] - 6s 22ms/step - loss: 0.0904 - accuracy: 0.9642 - val_loss: 0.6827 - val_accuracy: 0.8374
Epoch 4/15
269/269 [==============================] - 6s 23ms/step - loss: 0.0785 - accuracy: 0.9680 - val_loss: 0.7969 - val_accuracy: 0.8281
Epoch 5/15
269/269 [==============================] - 6s 22ms/step - loss: 0.0747 - accuracy: 0.9680 - val_loss: 0.6898 - val_accuracy: 0.8425
Epoch 6/15
269/269 [==============================] - 9s 32ms/step - loss: 0.0711 - accuracy: 0.9702 - val_loss: 0.9171 - val_accuracy: 0.8406
Epoch 7/15
269/269 [==============================] - 6s 23ms/step - loss: 0.0718 - accuracy: 0.9705 - val_loss: 0.9098 - val_accuracy: 0.8416

In [ ]:
########################################   GRU         #####################################################

In [ ]:
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(GRU(lstm_out, dropout=0.2, recurrent_dropout=0.2 ,return_sequences=True))
model.add(GRU(lstm_out, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 28, 128)          0         
 lDropout1D)                                                     
                                                                 
 gru (GRU)                   (None, 28, 196)           191688    
                                                                 
 gru_1 (GRU)                 (None, 196)               231672    
                                                                 
 dense_5 (Dense)             (None, 2)                 394       
                                                                 
Total params: 679,754
Trainable params: 679,754
Non-trainable params: 0
________________________________________________

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size,validation_data=(X_test,Y_test), verbose = 1)

In [ ]:
X.shape[1]

28

In [ ]:
########################################   Bidirectional LSTM        #####################################################

In [ ]:
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2 ,return_sequences=True)))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_10 (Spati  (None, 28, 128)          0         
 alDropout1D)                                                    
                                                                 
 bidirectional_1 (Bidirectio  (None, 28, 392)          509600    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 392)              923552    
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 2)                 786       
                                                     

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size,validation_data=(X_test,Y_test), verbose = 1)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 128  # Only consider the first 200 words of each movie review

x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
embed_dim = 128  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(28,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train,Y_train, batch_size=32, epochs=20, validation_data=(X_test, Y_test)
)

Epoch 1/20
269/269 [==============================] - 20s 70ms/step - loss: 0.4409 - accuracy: 0.8129 - val_loss: 0.3512 - val_accuracy: 0.8495
Epoch 2/20
269/269 [==============================] - 22s 82ms/step - loss: 0.3056 - accuracy: 0.8752 - val_loss: 0.3434 - val_accuracy: 0.8527
Epoch 3/20
269/269 [==============================] - 19s 69ms/step - loss: 0.2633 - accuracy: 0.8883 - val_loss: 0.3694 - val_accuracy: 0.8448
Epoch 4/20
269/269 [==============================] - 18s 69ms/step - loss: 0.2289 - accuracy: 0.9005 - val_loss: 0.3944 - val_accuracy: 0.8281
Epoch 5/20
269/269 [==============================] - 19s 69ms/step - loss: 0.2092 - accuracy: 0.9116 - val_loss: 0.5017 - val_accuracy: 0.8486
Epoch 6/20
269/269 [==============================] - 19s 69ms/step - loss: 0.1924 - accuracy: 0.9105 - val_loss: 0.5890 - val_accuracy: 0.8411
Epoch 7/20
269/269 [==============================] - 19s 69ms/step - loss: 0.1769 - accuracy: 0.9168 - val_loss: 0.5328 - val_accuracy: